# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Variables

In [ ]:
N_FEATURES_LIST = [3, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 94]
N_FEATURES_LIST = [3, 5, 10, 15, 20,]
N_FEATURES = 3

MUTUAL_INFORMATION = "MUTUAL_INFORMATION"
SPEARMAN = "SPEARMAN"

CLASSIFIERS = "Classifiers"
ACCURACY = "Accuracy"
N_FEATURES_NAME = "N_Features"
FEATURE_SELECTION = "Feature Selection"
DIVERSITY = "Diversity"
FUSION_CLASSIFIER = "Fusion"

### Variables file

In [ ]:
REAL_RETURN = "Real Return"
RISK = "Risk"

UNIQUE_CLASSIFIERS = "######## CLASSIFICADORES UNICOS ########"
CV = "######## CV ########"
SELECTION_CLASSIFIERS = "######## CLASSIFIERS SELECTION ########"
WHITOUT_DIVERSITY = "######## WHITOUT DIVERSITY ########"
BAGGING = "######## BAGGING ########"
ADABOOST = "######## ADABOOST ########"
FUSION = "######## FUSION ########"
END = "END"

RANDOM_FOREST = "RANDOM FOREST"
SVM = "SVM"
DECISION_TREE = "DECISION TREE"
NAIVE_BAYES = "NAIVE BAYES"
NEURAL_NETWORK = "NEURAL NETWORK"
LOGISTIC_REGRESSION = "LOGISTIC REGRESSION"
KNEIGHBORS= "KNEIGHBORS"
XGBOOST = "XGBOOST"

## Create Dataframes

In [ ]:
UNIQUE_COLUMNS = [N_FEATURES_NAME, FEATURE_SELECTION, CLASSIFIERS, ACCURACY]
DF_UNIQUE_REAL_RETURN = pd.DataFrame(columns=UNIQUE_COLUMNS)
DF_UNIQUE_RISK = pd.DataFrame(columns=UNIQUE_COLUMNS)

FUSION_COLUMNS = [N_FEATURES_NAME, DIVERSITY, FUSION_CLASSIFIER, ACCURACY]
DF_FUSION_REAL_RETURN = pd.DataFrame(columns=FUSION_COLUMNS)
DF_FUSION_RISK = pd.DataFrame(columns=FUSION_COLUMNS)

## Read File

In [ ]:
def addUniqueClassifiers(n_features:int, method:str, unique_classifiers:pd.DataFrame):
  global DF_UNIQUE_REAL_RETURN
  global DF_UNIQUE_RISK
  
  for key in unique_classifiers.keys():
    row_result = [n_features, method, key, unique_classifiers[key][0]]
    row_risk = [n_features, method, key, unique_classifiers[key][1]]

    temp_df = pd.DataFrame([row_result], columns=DF_UNIQUE_REAL_RETURN.columns)
    DF_UNIQUE_REAL_RETURN = pd.concat([DF_UNIQUE_REAL_RETURN, temp_df], ignore_index=True)

    temp_df = pd.DataFrame([row_risk], columns=DF_UNIQUE_RISK.columns)
    DF_UNIQUE_RISK = pd.concat([DF_UNIQUE_RISK, temp_df], ignore_index=True)

In [ ]:
def addFusion(n_features:int, fusion_list:list, df_base:pd.DataFrame):
  for result in fusion_list:
    row = [n_features, 
           str(result[0]).replace("[", "").replace("]", ""),  
           str(result[1]).replace("[", "").replace("]", ""),
           result[2]]
    
    temp_df = pd.DataFrame([row], columns=df_base.columns)
    df_base = pd.concat([df_base, temp_df], ignore_index=True)

  return df_base

In [ ]:
def readFile(selection_feature:str, n_features:int):
  global DF_FUSION_REAL_RETURN
  global DF_FUSION_RISK

  FILE_PATH = f"./files/{selection_feature}_{n_features}.txt"
  file = open(FILE_PATH, 'r')

  unique_classifiers = {}
  cv_classifiers = {}

  real_return_whiout = []
  risk_whiout = []

  real_return_bagging = []
  risk_bagging = []

  real_return_adaboost = []
  risk_adaboost = []

  real_return_fusion = []
  risk_fusion = []

  for line in file:
    if UNIQUE_CLASSIFIERS in line:
      line = file.readline()
      while CV not in line:
        line = line.strip()
        classifier = line.split(":")
        if classifier.__len__() > 1:
          unique_classifiers[classifier[0]] = eval(classifier[1])
        line = file.readline()
    if CV in line:
      line = file.readline()
      while SELECTION_CLASSIFIERS not in line:
        line = line.strip()
        line = file.readline()
    if WHITOUT_DIVERSITY in line:
      line = file.readline()
      while BAGGING not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_whiout.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_whiout.append(bg[1].split(', '))
        line = file.readline()
    if BAGGING in line:
      line = file.readline()
      while ADABOOST not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_bagging.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_bagging.append(bg[1].split(', '))
        line = file.readline()
    if ADABOOST in line:
      line = file.readline()
      while FUSION not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_adaboost.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_adaboost.append(bg[1].split(', '))
        line = file.readline()
    if FUSION in line:
      line = file.readline()
      while END not in line:
        line = line.strip().replace(":", "").split(" ")
        if line.__len__() > 1 and line[2] == "RealReturn":
          real_return_fusion.append((line[0], line[1], line[3]))
        if line.__len__() > 1 and line[2] == "Risk":
          risk_fusion.append((line[0], line[1], line[3]))
        line = file.readline()

  addUniqueClassifiers(n_features, selection_feature, unique_classifiers)
  DF_FUSION_REAL_RETURN = addFusion(n_features, real_return_fusion, DF_FUSION_REAL_RETURN)

In [ ]:
def queryByColumn(df:pd.DataFrame, column:str, value:str):
  return df[df[column] == value]

In [ ]:
for features in N_FEATURES_LIST:
  readFile(MUTUAL_INFORMATION, features)

In [ ]:
DF_FUSION_REAL_RETURN

In [ ]:
queryByColumn(DF_FUSION_REAL_RETURN, DIVERSITY, "ADABOOST")